In [1]:
import numpy as np
from scipy.optimize import linprog

# Define constraints and parameters
fixed_cost = 320000  # Fixed yearly costs
studio_var_cost = 720 * 12  # Monthly cost * 12 months
one_bed_var_cost = 1000 * 12

# Estimated Rent Prices (Based on Data)
studio_rent = 1800
one_bed_rent = 2500

# Annual Revenue Per Unit
studio_revenue = studio_rent * 12
one_bed_revenue = one_bed_rent * 12

# Profit Per Unit
studio_profit = studio_revenue - studio_var_cost
one_bed_profit = one_bed_revenue - one_bed_var_cost

# Objective Function: Maximize Profit (linprog minimizes, so we negate the coefficients)
c = [-studio_profit, -one_bed_profit]

# Constraints
A_ub = [[800, 1200],   # Floor space constraint
        [1, 1]]        # Total units constraint
b_ub = [80000, 85]     # Constraint limits

# Market Demand Constraints (Bounds)
x_bounds = (0, 48)  # Studios (max 48 based on demand cap)
y_bounds = (0, 57)  # 1-Bedrooms (max 57 based on demand cap)

# Solve Optimization Model
res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=[x_bounds, y_bounds], method="highs")

# Check if optimization was successful
if res.success:
    optimal_studio_units = int(round(res.x[0]))
    optimal_one_bed_units = int(round(res.x[1]))
    optimal_profit = round(-res.fun - fixed_cost, 2)  # Adjusting for minimization in linprog
    
    # Display Results
    print(f"Optimal Studio Units: {optimal_studio_units}")
    print(f"Optimal 1-Bedroom Units: {optimal_one_bed_units}")
    print(f"Expected Annual Profit: ${optimal_profit:,.2f}")
else:
    print("Optimization failed:", res.message)


Optimal Studio Units: 48
Optimal 1-Bedroom Units: 35
Expected Annual Profit: $926,080.00


In [4]:
#sensitvity analysis 
# Reload the Excel file and re-run the sensitivity analysis
import numpy as np
import pandas as pd
from scipy.optimize import linprog

# Define base case parameters
fixed_cost = 320000  # Fixed yearly costs
studio_var_cost = 720 * 12  # Monthly cost * 12 months
one_bed_var_cost = 1000 * 12

# Base Rent Prices
studio_rent_base = 1800
one_bed_rent_base = 2500

# Market Constraints
x_bounds = (0, 48)  # Studios (max 48 based on demand cap)
y_bounds = (0, 57)  # 1-Bedrooms (max 57 based on demand cap)

# Define sensitivity scenarios (varying rent prices)
studio_rent_range = np.arange(1600, 2100, 100)  # Vary studio rent from $1600 to $2000
one_bed_rent_range = np.arange(2300, 2800, 100)  # Vary 1-Bed rent from $2300 to $2700

# Storage for results
sensitivity_results = []

# Run sensitivity analysis
for studio_rent in studio_rent_range:
    for one_bed_rent in one_bed_rent_range:
        # Compute annual revenue per unit
        studio_revenue = studio_rent * 12
        one_bed_revenue = one_bed_rent * 12

        # Compute profit per unit
        studio_profit = studio_revenue - studio_var_cost
        one_bed_profit = one_bed_revenue - one_bed_var_cost

        # Optimization model
        c = [-studio_profit, -one_bed_profit]  # Minimize negative profit

        # Constraints
        A_ub = [[800, 1200], [1, 1]]
        b_ub = [80000, 85]

        # Solve Optimization
        res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=[x_bounds, y_bounds], method="highs")

        if res.success:
            optimal_studio_units = int(round(res.x[0]))
            optimal_one_bed_units = int(round(res.x[1]))
            optimal_profit = round(-res.fun - fixed_cost, 2)
        else:
            optimal_studio_units, optimal_one_bed_units, optimal_profit = None, None, None

        # Store results
        sensitivity_results.append([studio_rent, one_bed_rent, optimal_studio_units, optimal_one_bed_units, optimal_profit])

# Convert results to DataFrame
df_sensitivity = pd.DataFrame(sensitivity_results, columns=[
    "Studio Rent ($)", "1-Bed Rent ($)", "Optimal Studio Units", "Optimal 1-Bedroom Units", "Expected Annual Profit ($)"
])

# Display results
print("Top 10 Sensitivity Analysis Results")
print(df_sensitivity.head(10))  # Displays first 10 rows


Top 10 Sensitivity Analysis Results
   Studio Rent ($)  1-Bed Rent ($)  Optimal Studio Units  \
0             1600            2300                    48   
1             1600            2400                    14   
2             1600            2500                    14   
3             1600            2600                    14   
4             1600            2700                    14   
5             1700            2300                    48   
6             1700            2400                    48   
7             1700            2500                    14   
8             1700            2600                    14   
9             1700            2700                    14   

   Optimal 1-Bedroom Units  Expected Annual Profit ($)  
0                       35                    727680.0  
1                       57                    790720.0  
2                       57                    859120.0  
3                       57                    927520.0  
4                 

In [5]:
#monte carlo simulation 
# Reload the dataset and re-run the Monte Carlo simulation
import numpy as np
import pandas as pd

# Monte Carlo Simulation Parameters
num_simulations = 10000  # Number of random simulations

# Base values for costs and revenue
fixed_cost = 320000  # Fixed yearly costs
studio_var_cost = 720 * 12  # Monthly cost * 12 months
one_bed_var_cost = 1000 * 12

# Base rent assumptions with some variability (mean, standard deviation)
studio_rent_mean = 1800
one_bed_rent_mean = 2500
rent_std_dev = 150  # Allow small fluctuation in rents

# Base vacancy rates with some variability
vacancy_mean = 0.10  # 10% vacancy assumed on average
vacancy_std_dev = 0.03  # 3% standard deviation

# Generate random scenarios for rent and vacancy
simulated_studio_rents = np.random.normal(studio_rent_mean, rent_std_dev, num_simulations)
simulated_one_bed_rents = np.random.normal(one_bed_rent_mean, rent_std_dev, num_simulations)
simulated_vacancy_rates = np.random.normal(vacancy_mean, vacancy_std_dev, num_simulations)

# Storage for results
monte_carlo_results = []

for i in range(num_simulations):
    # Get random values for this iteration
    studio_rent = max(simulated_studio_rents[i], 1000)  # Ensure rents don't drop below 1000
    one_bed_rent = max(simulated_one_bed_rents[i], 1500)  # Ensure 1-bed rent stays reasonable
    vacancy_rate = max(min(simulated_vacancy_rates[i], 0.20), 0.05)  # Keep vacancy rate between 5% and 20%

    # Compute effective revenue considering vacancy
    studio_revenue = studio_rent * 12 * (1 - vacancy_rate)
    one_bed_revenue = one_bed_rent * 12 * (1 - vacancy_rate)

    # Compute profit per unit
    studio_profit = studio_revenue - studio_var_cost
    one_bed_profit = one_bed_revenue - one_bed_var_cost

    # Solve for optimal unit mix using our previous optimal results (48 Studios, 35 1-Bedrooms)
    optimal_studio_units = 48
    optimal_one_bed_units = 35

    # Compute total profit
    total_profit = (
        (optimal_studio_units * studio_profit) + (optimal_one_bed_units * one_bed_profit) - fixed_cost
    )

    # Store results
    monte_carlo_results.append([studio_rent, one_bed_rent, vacancy_rate, total_profit])

# Convert results to DataFrame
df_monte_carlo = pd.DataFrame(monte_carlo_results, columns=[
    "Studio Rent ($)", "1-Bed Rent ($)", "Vacancy Rate", "Expected Annual Profit ($)"
])

# Display results
print(df_monte_carlo)


      Studio Rent ($)  1-Bed Rent ($)  Vacancy Rate  \
0         1993.077149     2572.423339      0.113555   
1         1859.555457     2453.785270      0.059803   
2         1887.276585     2568.541856      0.057254   
3         1574.344573     2216.239052      0.122716   
4         1951.128949     2482.902476      0.134475   
...               ...             ...           ...   
9995      1673.615452     2541.567355      0.056733   
9996      2073.081722     2503.279919      0.122414   
9997      1756.779518     2392.339749      0.076548   
9998      1772.430219     2634.997404      0.119541   
9999      1574.261598     2376.671838      0.054735   

      Expected Annual Profit ($)  
0                  820660.406086  
1                  821286.656371  
2                  887135.594916  
3                  457415.456397  
4                  720585.944255  
...                          ...  
9995               761490.653536  
9996               815874.518743  
9997               70759